In [ ]:
import numpy as np
import cv2, tqdm, glob, os, random
from multiprocessing import Pool
import matplotlib.pyplot as plt
from PIL import Image
from typing import Sequence

from torchvision import transforms
import torch
import torchvision.transforms.functional as TF

In [ ]:
train_files = glob.glob('./dataset/train/good/*')
train_files = np.array(train_files)

def read_img(path) :
    img = cv2.imread(path)
    return img

with Pool(processes = 16) as p:
    train_pic = list(tqdm.tqdm(p.imap(read_img, train_files, chunksize=100), total = len(train_files)))
    
plt.subplot(2, 3, 1)
img = Image.fromarray(train_pic[0].astype(np.uint8))
plt.imshow(img)

print(train_pic[0].shape)
print(type(train_pic[0]))
to_tensor = transforms.ToPILImage()
temp = to_tensor(train_pic[0])
print(type(temp))

In [ ]:
class MyRotateTransform:
    def __init__(self, angles: Sequence[int]):
        self.angles = angles

    def __call__(self, x):
        angle = random.choice(self.angles)
        return TF.rotate(x, angle)

transform = transforms.Compose([
    transforms.ColorJitter(brightness=[0.9, 1.1], contrast=0.5, saturation=0.5), # , hue=0.25
    transforms.RandomOrder([
        transforms.RandomHorizontalFlip(p = 0.9), 
        transforms.RandomVerticalFlip(p = 0.9), 
        # transforms.RandomRotation([0, 90,180,270])
        MyRotateTransform( [0, 90, 180, 270])
    ]) 
])

temp = to_tensor(train_pic[0])
temp = transform(temp)
# temp.save('temp.jpg')
print(type(temp))
plt.subplot(1, 2, 1)
img = Image.fromarray(train_pic[0].astype(np.uint8))
plt.imshow(img)
plt.subplot(1, 2, 2)
plt.imshow(temp)

In [ ]:
np.random.seed(69)
choice = np.arange(len(train_pic))
choice = np.random.choice(choice, 500, )
for i in range(len(choice)) :
    temp = to_tensor(train_pic[i])
    temp = transform(temp)
    plt.imshow(temp)
    temp.save('./dataset/train/good/new_{}.jpg'.format(i))